# Read in netcdfs of anomalies for each member 
- take the mmm for the globe
- take out the regions and then take the lat lon mean and save as a netcdf
- take the llm AND mmm and save as a netcdf 

In [1]:
import xarray as xr, matplotlib.pyplot as plt
from importlib import reload # need to use this if I edit a function file
import os
import numpy as np
import pandas as pd
import cartopy.crs as ccrs # to add in continents and change map projections 
from matplotlib.colors import LinearSegmentedColormap # to change colour bar????
import dask.diagnostics # dask allows you to check how long something is taking to load
import climtas # needed to count event statistics with a specified duration

In [2]:
# import custom functions
import sys 
sys.path.append('/home/563/kb6999/Functions') # use this if the function file is in a different directory to the notebook

import frequently_used_functions as func
import plotting_functions as fplot
import model_functions as funcM
import reanalysis_functions as funcR

# scott way of opening files

In [3]:
# create a list of member names excluding member 70 cos that file is problematic 
members = [*range(1,70),*range(71,81)]

## monthly anomalies

In [4]:
# create a list of the paths
# tmp_paths = [f"/g/data/w48/kb6999/20CR_TMP_members/R_anom_Glob_TMP{m:02d}.nc" for m in members]
# pr_paths = [f"/g/data/w48/kb6999/20CR_PRATE_members/R_anom_Glob_PRATE{m:02d}.nc" for m in members]

## seasonal anomalies

In [5]:
# # create a list of the paths
tmp_paths = [f"/g/data/w48/kb6999/20CR_TMP_seasonal_members/R_season_anom_Glob_TMP{m:02d}.nc" for m in members]
# pr_paths = [f"/g/data/w48/kb6999/20CR_PRATE_members/R_anom_Glob_PRATE{m:02d}.nc" for m in members]

In [8]:
xr.open_dataset('/g/data/w48/kb6999/20CR_TMP_seasonal_members/R_season_anom_Glob_TMP01.nc')

<xarray.Dataset>
Dimensions:     (lat: 256, lon: 512, season: 4, seasonyear: 167)
Coordinates:
  * season      (season) object 'DJF' 'JJA' 'MAM' 'SON'
  * lon         (lon) float32 0.0 0.703 1.406 2.109 ... 357.1 357.8 358.5 359.2
  * lat         (lat) float32 89.46 88.77 88.07 87.37 ... -88.07 -88.77 -89.46
  * seasonyear  (seasonyear) int64 1850 1851 1852 1853 ... 2013 2014 2015 2016
Data variables:
    TMP         (seasonyear, season, lat, lon) float32 ...

In [11]:
# open all members of temperature 
ds_tmp = xr.open_mfdataset(tmp_paths, combine='nested', concat_dim='member', chunks={'seasonyear': 50})
ds_tmp.coords['member'] = members
ds_tmp

,Array,Chunk
Bytes,27.67 GB,104.86 MB
Shape,"(79, 167, 4, 256, 512)","(1, 50, 4, 256, 512)"
Count,1027 Tasks,316 Chunks
Type,float32,numpy.ndarray


In [12]:
# rename variable
reanal = xr.Dataset({'tmp': ds_tmp.TMP})#, 'pr': ds_pr.PRATE})
reanal

,Array,Chunk
Bytes,27.67 GB,104.86 MB
Shape,"(79, 167, 4, 256, 512)","(1, 50, 4, 256, 512)"
Count,1027 Tasks,316 Chunks
Type,float32,numpy.ndarray


## area weighting and landmask

In [13]:
# area weighting 
reanal_w = reanal*np.cos(reanal.lat*(np.pi/180))
reanal_w

,Array,Chunk
Bytes,27.67 GB,104.86 MB
Shape,"(79, 167, 4, 256, 512)","(1, 50, 4, 256, 512)"
Count,1344 Tasks,316 Chunks
Type,float32,numpy.ndarray


In [14]:
monthly_anom_glob = reanal_w

In [15]:
# import landmask dataset 
landfrac_ds = xr.open_dataset('/g/data/w48/kb6999/20CR_data_netcdfs/land_20CR.nc')
landmask = landfrac_ds

In [16]:
# Select out SH and NH anomalies 
monthly_anom_SH = monthly_anom_glob.sel(lat=slice(0,-90)) 
monthly_anom_NH = monthly_anom_glob.sel(lat=slice(90,0)) 
# select out the Australian and E Australian anomalies 
with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    monthly_anom_Aus = monthly_anom_glob.sel(lat=slice(-10,-50), lon=slice(110,160)).where(landmask.LAND==1.0, drop=True)
    monthly_anom_EA = monthly_anom_glob.sel(lat=slice(-10,-50), lon=slice(140,155)).where(landmask.LAND==1.0, drop=True)

In [17]:
monthly_anom_Aus

,Array,Chunk
Bytes,529.41 MB,105.60 kB
Shape,"(79, 167, 4, 44, 57)","(1, 50, 4, 11, 12)"
Count,15921 Tasks,6320 Chunks
Type,float32,numpy.ndarray


## means and percentiles

In [18]:
# take the multi-member mean
mmm_anom_glob = monthly_anom_glob.mean(dim=['member'])

In [19]:
# take lat lon mean
llm_Glob = monthly_anom_glob.mean(dim=['lat','lon'])
llm_SH = monthly_anom_SH.mean(dim=['lat','lon'])
llm_NH = monthly_anom_NH.mean(dim=['lat','lon'])
llm_Aus = monthly_anom_Aus.mean(dim=['lat','lon'])
llm_EA = monthly_anom_EA.mean(dim=['lat','lon'])

In [20]:
llm_Glob

<xarray.Dataset>
Dimensions:     (member: 79, season: 4, seasonyear: 167)
Coordinates:
  * season      (season) object 'DJF' 'JJA' 'MAM' 'SON'
  * seasonyear  (seasonyear) int64 1850 1851 1852 1853 ... 2013 2014 2015 2016
  * member      (member) int64 1 2 3 4 5 6 7 8 9 ... 72 73 74 75 76 77 78 79 80
Data variables:
    tmp         (member, seasonyear, season) float32 dask.array<chunksize=(1, 50, 4), meta=np.ndarray>

In [21]:
# calculate the percentiles and then take the mean along the model dimension for monthly anomalies
p10_mon_Glob = llm_Glob.chunk({'member': -1}).quantile(0.1, dim=['member'])
p10_mon_NH = llm_NH.chunk({'member': -1}).quantile(0.1, dim=['member'])
p10_mon_SH = llm_SH.chunk({'member': -1}).quantile(0.1, dim=['member'])
p10_mon_Aus = llm_Aus.chunk({'member': -1}).quantile(0.1, dim=['member'])
p10_mon_EA = llm_EA.chunk({'member': -1}).quantile(0.1, dim=['member'])
# calculate the percentiles and then take the mean along the model dimension for monthly anomalies
p90_mon_Glob = llm_Glob.chunk({'member': -1}).quantile(0.9, dim=['member'])
p90_mon_NH = llm_NH.chunk({'member': -1}).quantile(0.1, dim=['member'])
p90_mon_SH = llm_SH.chunk({'member': -1}).quantile(0.9, dim=['member'])
p90_mon_Aus = llm_Aus.chunk({'member': -1}).quantile(0.9, dim=['member'])
p90_mon_EA = llm_EA.chunk({'member': -1}).quantile(0.9, dim=['member'])

In [22]:
# Take the multi-member mean for each region
mmm_mon_Glob = llm_Glob.mean(dim='member')
mmm_mon_NH = llm_NH.mean(dim='member')
mmm_mon_SH = llm_SH.mean(dim='member')
mmm_mon_Aus = llm_Aus.mean(dim='member')
mmm_mon_EA = llm_EA.mean(dim='member')

## Write to netcdf

# choose monthly or seasonal data!! 

In [23]:
# monthly
# path = '/g/data/w48/kb6999/20CR_data_for_plots/'
# seasonal 
path = '/g/data/w48/kb6999/20CR_TMP_seasonal_data_for_plots/'

In [24]:
with dask.diagnostics.ProgressBar():
    mmm_anom_glob.to_netcdf(f'{path}mmm_anom_glob.nc')

[########################################] | 100% Completed |  1min 26.8s


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.01/lib/python3.8/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


In [25]:
with dask.diagnostics.ProgressBar():
    mmm_mon_Glob.to_netcdf(f'{path}mmm_mon_Glob.nc')
    mmm_mon_NH.to_netcdf(f'{path}mmm_mon_NH.nc')
    mmm_mon_SH.to_netcdf(f'{path}mmm_mon_SH.nc')
    mmm_mon_Aus.to_netcdf(f'{path}mmm_mon_Aus.nc')
    mmm_mon_EA.to_netcdf(f'{path}mmm_mon_EA.nc')

[##############################          ] | 75% Completed | 40.6s

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.01/lib/python3.8/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


[########################################] | 100% Completed | 45.3s
[##############################          ] | 75% Completed | 43.4s

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.01/lib/python3.8/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


[########################################] | 100% Completed | 47.3s
[##############################          ] | 76% Completed | 29.2s

...

[########################################] | 100% Completed | 32.9s
[##############################          ] | 76% Completed | 28.6s

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.01/lib/python3.8/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


[########################################] | 100% Completed | 33.4s
[##############################          ] | 76% Completed | 37.6s

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.01/lib/python3.8/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


[########################################] | 100% Completed | 42.2s


In [26]:
with dask.diagnostics.ProgressBar():
    llm_Glob.to_netcdf(f'{path}llm_mon_Glob.nc')
    llm_SH.to_netcdf(f'{path}llm_mon_NH.nc') 
    llm_NH.to_netcdf(f'{path}llm_mon_SH.nc')
    llm_Aus.to_netcdf(f'{path}llm_mon_Aus.nc')
    llm_EA.to_netcdf(f'{path}llm_mon_EA.nc')

[                                        ] | 1% Completed |  0.6s

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.01/lib/python3.8/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


[########################################] | 100% Completed | 35.2s
[                                        ] | 1% Completed |  0.5s

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.01/lib/python3.8/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


[########################################] | 100% Completed | 47.2s
[                                        ] | 1% Completed |  0.5s

...

[########################################] | 100% Completed | 38.2s
[#                                       ] | 2% Completed |  1.1s

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.01/lib/python3.8/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


[########################################] | 100% Completed | 47.1s
[                                        ] | 1% Completed |  1.1s

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.01/lib/python3.8/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


[########################################] | 100% Completed | 32.9s


In [27]:
with dask.diagnostics.ProgressBar():
    p10_mon_Glob.to_netcdf(f'{path}p10_mon_Glob.nc')
    p10_mon_NH.to_netcdf(f'{path}p10_mon_NH.nc')
    p10_mon_SH.to_netcdf(f'{path}p10_mon_SH.nc')
    p10_mon_Aus.to_netcdf(f'{path}p10_mon_Aus.nc')
    p10_mon_EA.to_netcdf(f'{path}p10_mon_EA.nc')

[##############################          ] | 76% Completed | 27.9s

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.01/lib/python3.8/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


[########################################] | 100% Completed | 31.5s


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.01/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1389: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,


[##############################          ] | 76% Completed | 26.0s

...

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.01/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1389: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,


[########################################] | 100% Completed | 32.4s
[##############################          ] | 76% Completed | 24.2s

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.01/lib/python3.8/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


[########################################] | 100% Completed | 29.2s


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.01/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1389: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,


In [28]:
with dask.diagnostics.ProgressBar():
    p90_mon_Glob.to_netcdf(f'{path}p90_mon_Glob.nc')
    p90_mon_NH.to_netcdf(f'{path}p90_mon_NH.nc')
    p90_mon_SH.to_netcdf(f'{path}p90_mon_SH.nc')
    p90_mon_Aus.to_netcdf(f'{path}p90_mon_Aus.nc')
    p90_mon_EA.to_netcdf(f'{path}p90_mon_EA.nc')

[##############################          ] | 75% Completed | 39.8s

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.01/lib/python3.8/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


[########################################] | 100% Completed | 43.2s


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.01/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1389: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,


[##############################          ] | 75% Completed | 30.4s

...

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.01/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1389: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,


[########################################] | 100% Completed |  1min  8.7s
[##############################          ] | 76% Completed | 59.8s

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.01/lib/python3.8/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


[########################################] | 100% Completed |  1min  5.9s


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.01/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1389: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,


In [29]:
mmm = xr.open_dataset(f'{path}mmm_mon_Aus.nc')
mmm

<xarray.Dataset>
Dimensions:     (season: 4, seasonyear: 167)
Coordinates:
  * season      (season) object 'DJF' 'JJA' 'MAM' 'SON'
  * seasonyear  (seasonyear) int64 1850 1851 1852 1853 ... 2013 2014 2015 2016
Data variables:
    tmp         (seasonyear, season) float32 ...